Round 4¶
fit a Random forest Classifier on the data and compare the accuracy.

tune the hyper paramters with gridsearch and check the results.

Managing imbalance in the dataset

Check for the imbalance. Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.

Each time fit the model and see how the accuracy of the model is.

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,  ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


import warnings
warnings.filterwarnings("ignore")

In [2]:
url = 'DATA_Customer_Churn.csv'

churnData = pd.read_csv(url)

In [3]:
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [4]:
churnData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   OnlineSecurity    7043 non-null   object 
 7   OnlineBackup      7043 non-null   object 
 8   DeviceProtection  7043 non-null   object 
 9   TechSupport       7043 non-null   object 
 10  StreamingTV       7043 non-null   object 
 11  StreamingMovies   7043 non-null   object 
 12  Contract          7043 non-null   object 
 13  MonthlyCharges    7043 non-null   float64
 14  TotalCharges      7043 non-null   object 
 15  Churn             7043 non-null   object 
dtypes: float64(1), int64(2), object(13)
memory

In [5]:
# Filling missing values

churnData.loc[churnData['TotalCharges'] == ' ', 'TotalCharges'] = churnData['MonthlyCharges']


In [6]:
# Convert total charges to float

churnData['TotalCharges'] = churnData['TotalCharges'].astype(float)

In [7]:
# Converting Churn to numerical value

y = churnData['Churn'].map({'Yes': 1, 'No': 0})

In [8]:
y.value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

In [9]:
X = churnData.drop('Churn', axis=1)

In [10]:
num_df = X.select_dtypes(include='number')
cat_df = X.select_dtypes(exclude='number')

In [11]:
cat_df['Contract'].value_counts()

Month-to-month    3875
Two year          1695
One year          1473
Name: Contract, dtype: int64

In [12]:
df_to_encode = cat_df['Contract']

In [16]:
cat_df_encoded = pd.get_dummies(cat_df)

In [19]:
final_df = pd.concat([num_df, cat_df_encoded], axis=1)

In [20]:
final_df

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,...,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year
0,0,1,29.85,29.85,1,0,0,1,1,0,...,0,1,0,0,1,0,0,1,0,0
1,0,34,56.95,1889.50,0,1,1,0,1,0,...,0,1,0,0,1,0,0,0,1,0
2,0,2,53.85,108.15,0,1,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0
3,0,45,42.30,1840.75,0,1,1,0,1,0,...,1,1,0,0,1,0,0,0,1,0
4,0,2,70.70,151.65,1,0,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,24,84.80,1990.50,0,1,0,1,0,1,...,1,0,0,1,0,0,1,0,1,0
7039,0,72,103.20,7362.90,1,0,0,1,0,1,...,0,0,0,1,0,0,1,0,1,0
7040,0,11,29.60,346.45,1,0,0,1,0,1,...,0,1,0,0,1,0,0,1,0,0
7041,1,4,74.40,306.60,0,1,0,1,1,0,...,0,1,0,0,1,0,0,1,0,0


In [23]:
X_train, X_test, y_train, y_test = train_test_split(final_df, y, test_size=0.2, random_state=42)

In [24]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train, y_train = smote.fit_resample(X_train, y_train)
y_train.value_counts()

0    4138
1    4138
Name: Churn, dtype: int64

In [25]:
from sklearn.ensemble import RandomForestClassifier

rfc_ops = {"max_depth":6,
           "min_samples_leaf":20,
           "n_estimators":30,
           "bootstrap":True,
           "oob_score":True,
           "random_state":42}

clf = RandomForestClassifier(**rfc_ops)

        #max_depth=6,min_samples_leaf=20,max_features=None,n_estimators=100,
         #                    bootstrap=True,oob_score=True, random_state=RAND_STATE)
clf.fit(X_train, y_train)
print("train prediction accuracy score: %.2f" %(clf.score(X_train, y_train)))
print("test prediction accuracy score: %.2f"  %(clf.score(X_test, y_test)))

train prediction accuracy score: 0.83
test prediction accuracy score: 0.77


In [26]:
clf.oob_score_

0.819961333977767

In [27]:
from sklearn.model_selection import GridSearchCV

In [36]:
param_grid = {
    'n_estimators': [50, 100],
    'min_samples_split': [80],
    'min_samples_leaf' : [50],
    'max_features': ['sqrt'],
    ##'max_samples' : ['None', 0.5],
   'max_depth':[2,7],
    'bootstrap':[True,False]
    }

In [37]:
grid_search = GridSearchCV(clf, param_grid, cv=5,return_train_score=True,n_jobs=-1,)

In [38]:
grid_search.fit(X_train,y_train.values.ravel())

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'bootstrap': [True, False], 'max_depth': [2, 7],
                         'max_features': ['sqrt'], 'min_samples_leaf': [50],
                         'min_samples_split': [80], 'n_estimators': [50, 100]},
             return_train_score=True)

In [39]:
best_params = grid_search.best_params_ #To check the best set of parameters returned
best_params

{'bootstrap': False,
 'max_depth': 7,
 'max_features': 'sqrt',
 'min_samples_leaf': 50,
 'min_samples_split': 80,
 'n_estimators': 100}

In [35]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.436637,0.010624,0.032412,0.004602,3,sqrt,50,80,50,"{'max_depth': 3, 'max_features': 'sqrt', 'min_...",...,0.774895,0.023098,4,0.804834,0.798218,0.770427,0.774656,0.774656,0.784558,0.014096
1,0.881466,0.020377,0.059193,0.007537,3,sqrt,50,80,100,"{'max_depth': 3, 'max_features': 'sqrt', 'min_...",...,0.779487,0.025082,3,0.803323,0.803806,0.773297,0.773448,0.777526,0.786280,0.014195
2,0.548725,0.006301,0.039244,0.004569,5,sqrt,50,80,50,"{'max_depth': 5, 'max_features': 'sqrt', 'min_...",...,0.801360,0.040597,2,0.837764,0.828123,0.798067,0.803655,0.800483,0.813618,0.016168
3,0.907720,0.067600,0.057472,0.012973,5,sqrt,50,80,100,"{'max_depth': 5, 'max_features': 'sqrt', 'min_...",...,0.805710,0.041530,1,0.841088,0.832956,0.802749,0.808186,0.804410,0.817878,0.015939


In [40]:
rfc_ops = {'bootstrap': False,
           'max_depth': 7,
           'max_features': 'sqrt',
           'min_samples_leaf': 50,
           'min_samples_split': 80,
           'n_estimators': 100}

clf = RandomForestClassifier(**rfc_ops)

        #max_depth=6,min_samples_leaf=20,max_features=None,n_estimators=100,
         #                    bootstrap=True,oob_score=True, random_state=RAND_STATE)
clf.fit(X_train, y_train)
print("train prediction accuracy score: %.2f" %(clf.score(X_train, y_train)))
print("test prediction accuracy score: %.2f"  %(clf.score(X_test, y_test)))

train prediction accuracy score: 0.83
test prediction accuracy score: 0.78
